<a href="https://colab.research.google.com/github/MarcosRolando/OrgaDeDatosTP2/blob/main/neuralNetwoork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import feature_column


In [2]:
df = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')
df = df[df['Stage'].isin(['Closed Won', 'Closed Lost'])]
df.loc[:, 'Stage'].replace({'Closed Won':1, 'Closed Lost':0}, inplace=True) #0 corresponde a que el caso fue Closed Lost, 1 a que fue Closed Won. Asi tenemos un problema de clasificacion binario que puede entender la red neuronal.

df.loc[:, 'Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], 'coerce',
                                                                format='%m/%d/%Y')
df.loc[:, 'Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], 'coerce',
                                                                                    format='%m/%d/%Y')
df = df[df['Opportunity_ID'] != 9773] #Hardcodeo este filtrado porque el id 9773 tiene mal cargada la fecha de delviery end, dando una diferencia de 200 anios xd"

#Pongo .loc porque pandas me jode con warnings que son falsos positivos de slice copy"
#Gracias Pandas!"

#Creamos una nueva columna (Feature Engineering) que contiene la longitud en dias 
#estimada de la operacion. En el informe habiamos encontrado que aparentaba haber
#una relacion cuadratica de decrecimiento a medida que aumentaban los dias donde disminuia
#la chance de completar la operacion.
df['Delta_Time'] = df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']
df.loc[:, 'Delta_Time'] = df['Delta_Time'].dt.days

In [3]:

#Borro columnas que tengan el mismo dato en todas las entradas, o inconsecuentes como el ID / Opportunity_ID
#Algunas columnas borradas son porque pienso que no tienen incidencia, ir viendo.
#TODO: Analizar si el Sales_Contract_No no es que importe el numero en si, sino si tiene
#o no tiene numero de contrato. Por ahora no lo meto como input.
#TODO: Ver el mismo tema con la columna 'Price', la mayoria tiene None u Other
#y solo unos pocos tienen precio numerico. Quiza importe que tenga precio o no tenga,
#o si no tiene precio quiza importe si es None u Other. Por ahora no lo pongo
#como input.
df.drop(columns=['Submitted_for_Approval', 'Last_Activity', 'ASP_(converted)_Currency', 
                 'Prod_Category_A', 'ID', 'Opportunity_ID', 'Sales_Contract_No',
                 'Price','ASP','ASP_Currency','Total_Amount_Currency','Total_Amount'],inplace=True)

#Renombramos las columnas que tienen caracteres que TensorFlow no acepta como validos.
#Estos particularmente son whitespace, coma y parentesis por ejemplo.
df.rename(columns={'ASP_(converted)':'ASP_converted','Pricing, Delivery_Terms_Quote_Appr':
                   'Pricing_Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved':
                   'Pricing_Delivery_Terms_Approved','Source ':'Source'},inplace=True)

#Columnas que consideramos numericas
numeric_columns = ['Pricing_Delivery_Terms_Quote_Appr','Pricing_Delivery_Terms_Approved',
                     'Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved',
                     'ASP_converted','TRF','Total_Amount','Total_Taxable_Amount']

#Columnas que consideramos clasificatorias con rango numerico
bucketized_columns = ['Delta_Time']

#Columnas que consideramos categoricas de pocos valores posibles
indicator_columns = ['Region','Territory','Bureaucratic_Code','Source',
                       'Billing_Country','Account_Name','Opportunity_Name',
                       'Account_Owner','Opportunity_Owner','Account_Type',
                       'Opportunity_Type','Quote_Type','Delivery_Terms',
                       'Brand','Product_Type','Size','Product_Category_B',
                       'Currency','Last_Modified_By','Product_Family',
                       'Product_Name','Total_Amount_Currency',
                       'Total_Taxable_Amount_Currency']

#Drop columnas que vamos a usar pero por ahora no las uso
df.drop(columns=['Account_Created_Date','Opportunity_Created_Date',
                 'Quote_Expiry_Date','Last_Modified_Date',
                 'Planned_Delivery_Start_Date','Planned_Delivery_End_Date',
                 'Month','Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date'],inplace=True)

#Debemos separar algunos de los registros para armar un set de test propio (no el de la catedra). De esta forma sabremos rapidamente
#si nuestro modelo esta dando resultados optimos o no sin necesidad de estar subiendo el TP a Kaggle constantemente.
#Sin embargo, no queremos usar tantos registros ya que estariamos disminuyendo el set de entrenamiento considerablemente.
#Podemos empezar reservando 2000 registros para el test de prueba y ver que onda. Pasariamos de tener 16 mil a 14 mil 
#registros para el set de entrenamiento, no es una perdida importantisima creo en principio, asi que arrancamos con eso.

#Por otro lado, nuestro test de prueba deberia tener un 50 50 de Closed Won y Closed Lost, por lo que no podemos elegir asi nomas
#al azar.

#TODO: DEJO COMO TAREA ARMAR EL SET DE PRUEBA, POR AHORA SOLO NOS PREOCUPAMOS DE LOGRAR HACER ANDAR LA RED NEURONAL.

df.head()

,Region,Territory,Pricing_Delivery_Terms_Quote_Appr,Pricing_Delivery_Terms_Approved,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Source,Billing_Country,Account_Name,Opportunity_Name,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Brand,Product_Type,Size,Product_Category_B,Currency,Last_Modified_By,Product_Family,Product_Name,ASP_converted,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Delta_Time
0,EMEA,None,1,1,1,1,Bureaucratic_Code_4,None,Netherlands,Account_Name_619,Opportunity_Name_12598,Person_Name_51,Person_Name_18,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,None,None,None,None,None,Person_Name_18,Product_Family_77,Product_Name_99,0.58817,10,EUR,5272800.0,EUR,5272800.0,0,60.0
1,EMEA,None,0,0,0,0,Bureaucratic_Code_4,None,Netherlands,Account_Name_619,Opportunity_Name_12600,Person_Name_51,Person_Name_20,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,None,None,None,None,None,Person_Name_20,Product_Family_77,Product_Name_100,0.59948,0,EUR,48230.0,EUR,48230.0,1,2.0
2,Americas,NW America,0,0,0,0,Bureaucratic_Code_4,Source_7,United States,Account_Name_1794,Opportunity_Name_469,Person_Name_64,Person_Name_8,Account_Type_5,Opportunity_Type_1,Non Binding,Delivery_Terms_4,None,None,None,None,None,Person_Name_8,Product_Family_81,Product_Name_91,0.48000,0,USD,83865.6,USD,83865.6,1,0.0
3,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,Source_11,United States,Account_Name_1201,Opportunity_Name_415,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,Other,Product_Type_0,Size_4,Product_Category_B_16,USD,Person_Name_8,Product_Family_209,Product_Name_432,0.53000,14,USD,7421881.5,USD,7421881.5,0,58.0
4,Americas,NW America,1,0,1,0,Bureaucratic_Code_5,Source_11,United States,Account_Name_1201,Opportunity_Name_851,Person_Name_8,Person_Name_8,Account_Type_5,Opportunity_Type_19,Non Binding,Delivery_Terms_1,Other,Product_Type_0,Size_4,Product_Category_B_16,USD,Person_Name_8,Product_Family_209,Product_Name_432,0.53000,25,USD,13357192.5,USD,13357192.5,0,27.0


In [13]:
# Metodo que pasa de DataFrame de Pandas a un DataSet de TensorFlow
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Stage') #Retorna la columna Stage, eliminandolo simultaneamente del DataFrame. 'Stage' seria nuestra columna 'target', es decir, lo que queremos predecir.
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels)) #Crea un DataSet cuyos elementos son slices de los tensores pasados a la funcion. Ver documentacion oficial para comprender bien.
  #En pocas palabras, le pasamos las columnas con los datos como diccionario (estilo 'columna':[dato1,dato2,dato3]) y una lista con los resultados estilo [resu1,resu2,resu3].
  #Se genera un DataSet del estilo [('columna':[dato1,dato2,dato3], [resu1, resu2, resu3])].
  #En realidad 'columna' es una lista de todas las columnas con sus correspondientes datos, pero se entiende la idea creo.
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe)) #Al tener el buffer_size del mismo tamanio que la cantidad de datos del dataset, tenemos perfect shuffling (ver documentacion para comprender).
    #Basicamente mezlcamos el dataset para que luego los batches que se armen contengan distintos elementos si lo entrenamos distintas veces.
  ds = ds.batch(batch_size) #Arma batches de tamanio batch_size entre elementos consecutivos del DataSet.
  return ds

In [14]:
#Arma las features, asignando las columnas del DataFrame segun corresponda al tipo de feature
#(numerico, categorico, etc). Entiendase por feature a las columnas del DataFrame.
def set_up_feature_columns(dataframe, numeric_columns, indicator_columns):
  features = []

  #numeric features
  for column_name in numeric_columns:
    features.append(feature_column.numeric_column(column_name))

  # bucketized cols
  boundaries = []
  for i in range(35):
    boundaries.append(i*10.0)

  estimatedOpportunityTimeLapse = feature_column.numeric_column('Delta_Time')
  time_buckets = feature_column.bucketized_column(estimatedOpportunityTimeLapse, 
                                                  boundaries)
  features.append(time_buckets)

  # indicator_columns (one-hot value vector, para aquellas columnas categoricas de pocas opciones)
  for column_name in indicator_columns:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
                                          column_name, dataframe[column_name].unique())
    indicator_column = feature_column.embedding_column(categorical_column, dimension=8)
    features.append(indicator_column)

  return features

Preparamos los features para el modelo, es decir, seteamos cada una de las columnas que vayamos a utilizar del DataFrame. Luego generamos el DataSet en base al DataFrame para darselo como input al modelo.

In [15]:
  features = set_up_feature_columns(df,numeric_columns,indicator_columns)
  feature_layer = tf.keras.layers.DenseFeatures(features)
  ds = df_to_dataset(df,batch_size=32)

[0.0, 20.0, 40.0, 60.0, 80.0, 100.0, 120.0, 140.0, 160.0, 180.0, 200.0, 220.0, 240.0, 260.0, 280.0, 300.0]


Creamos y compilamos el modelo. En esta seccion se tunean las propiedades del modelo.

In [16]:
model = tf.keras.Sequential([
  feature_layer,
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(512,activation='relu'),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(512,activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

Entrenamos el modelo

In [17]:
#Ignoren el WARNING, esta en la documentacion tambien. Nadie le da bola en StackOverflow xd.
model.fit(ds,epochs=150)

Epoch 1/200
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
132/132 [==============================] - 3s 12ms/step - loss: 8.7291 - accuracy: 0.4340
Epoch 2/200
132/132 [==============================] - 2s 12ms/step - loss: 8.7234 - accuracy: 0.4345
Epoch 3/200
132/132 [==============================] - 2s 12ms/step - loss: 8.6551 - accuracy: 0.4389
Epoch 4/200
132/132 [==============================] - 2s 12ms/step - loss: 8.6747 - accuracy: 0.4376
Epoch 5/200
132/132 [==============================] - 2s 12ms/step - loss: 8.8157 - accuracy: 0.4285
Epoch 6/200
132/132 [==============================] - 2s 12ms/step - loss: 8.6799 - accuracy: 0.4373
Epoch 7/200
132/132 [==============================] - 2s 12ms/step - loss: 8.7673 - accuracy: 0.4316
Epoch 8/200
132/132 [==============================] - 2s 12ms/step - loss: 8.7721 - accuracy: 0.4313
Epoch 9/200
132/132 [==============================] - 2s 12ms/step - loss

KeyboardInterrupt: ignored

Evaluamos el modelo

In [ ]:
loss, accuracy = model.evaluate(ds)
print("Accuracy", accuracy)